In [3]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
#os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
#import numpy as np
#import torch

#from od3d.cv.io import read_image
#from od3d.cv.visual.show import show_img
from pathlib import Path

import matplotlib.pyplot as plt
from PIL import Image
import cv2 
# `video_dir` a directory of JPEG frames with filenames like `<frame_index>.jpg`
video_dir = "/data/lmbraid19/sommerl/datasets/Objectron/frames/bike/batch_0_2/"
video_dir = "/home/sommerl/video/batch_0_0/"
# scan all the JPEG frame names in this directory
frame_names = [
    p for p in os.listdir(video_dir)
    if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG"]
]
frame_names.sort(key=lambda p: int(os.path.splitext(p)[0]))

frame_idx = 0
fpath = os.path.join(video_dir, frame_names[frame_idx])
print(fpath)

#img = Image.open(fpath)
#img = read_image(fpath)
#show_img(img)

#img.shape
# 

/home/sommerl/video/batch_0_0/0.jpg


In [4]:
import torch
from sam2.sam2_video_predictor import SAM2VideoPredictor

predictor = SAM2VideoPredictor.from_pretrained("facebook/sam2-hiera-large")
mask_generator = SAM2AutomaticMaskGenerator.from_pretrained("facebook/sam2-hiera-large")


In [7]:
inference_state = predictor.init_state(video_path=video_dir)

frame loading (JPEG): 100%|██████████| 318/318 [00:18<00:00, 17.36it/s]


In [4]:
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
import torch
mask_generator = SAM2AutomaticMaskGenerator.from_pretrained("facebook/sam2-hiera-large", device=torch.device("cuda"))

In [2]:
mask_generator.cuda()

AttributeError: 'SAM2AutomaticMaskGenerator' object has no attribute 'cuda'

In [ ]:
masks = mask_generator.generate(img.permute(2, 0, 1).numpy())


In [20]:
import torch
from sam2.sam2_image_predictor import SAM2ImagePredictor

predictor = SAM2ImagePredictor.from_pretrained("facebook/sam2-hiera-large")



In [4]:
import torch
from sam2.sam2_video_predictor import SAM2VideoPredictor

predictor = SAM2VideoPredictor.from_pretrained("facebook/sam2-hiera-large")

with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    state = predictor.init_state(video_dir)

frame loading (JPEG): 100%|██████████| 318/318 [00:17<00:00, 18.07it/s]


In [ ]:
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):

    # add new prompts and instantly get the output on the same frame
    frame_idx, object_ids, masks = predictor.add_new_points_or_box(state, <your_prompts>):

In [5]:
import torch
from sam2.sam2_video_predictor import SAM2VideoPredictor
import numpy as np

predictor = SAM2VideoPredictor.from_pretrained("facebook/sam2-hiera-large")
masks_all = []
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    state = predictor.init_state(video_dir)

    # add new prompts and instantly get the output on the same frame
    frame_idx, object_ids, masks = predictor.add_new_points_or_box(state, 
                        frame_idx=0, 
                        obj_id=0, points=np.array([[720, 960]]), labels=np.array([1]))

    # propagate the prompts to get masklets throughout the video
    for frame_idx, object_ids, masks in predictor.propagate_in_video(state):
        masks_all.append(masks)

frame loading (JPEG): 100%|██████████| 318/318 [00:17<00:00, 17.85it/s]
/home/sommerl/PycharmProjects/od3d/venv_od3d/lib/python3.10/site-packages/sam2/sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (/home/sommerl/PycharmProjects/od3d/venv_od3d/lib/python3.10/site-packages/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 318/318 [00:24<00:00, 13.07it/s]


In [16]:
out_dir = Path("/home/sommerl/video/out_sam2_cu12")
out_dir.mkdir(exist_ok=True)
for i in range(len(masks_all)):
    img = ((masks_all[i][0][0] > 0.5) * 255).detach().cpu().numpy().astype(np.uint8)
    fpath = out_dir.joinpath(f"{i}.png")
    cv2.imwrite(str(fpath), img)
    
    # show_img(masks_all[i][0] > 0.5, fpath=out_dir.joinpath(f"{i}.png"))

In [15]:
img.dtype

dtype('uint8')

In [12]:
import numpy as np
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    predictor.set_image(img.permute(1, 2, 0).contiguous().detach().cpu().numpy(),)
    # masks, _, _ = predictor.predict() # <input_prompts>
    masks, scores, logits = predictor.predict(
        point_coords=np.array([[720, 960]]),
        point_labels=np.array([1]),
        multimask_output=True,
    )
    mask = masks[scores.argmax()]

/home/sommerl/PycharmProjects/od3d/venv_od3d/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [17]:
mask.shape

(1920, 1440)

In [18]:
show_img(torch.from_numpy(mask[None,]))

32

In [1]:
masks.shape

NameError: name 'img' is not defined

In [8]:
# run propagation throughout the video and collect the results in a dict
video_segments = {}  # video_segments contains the per-frame segmentation results
for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
    video_segments[out_frame_idx] = {
        out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }

RuntimeError: No points are provided; please add points first

In [10]:
inference_state.keys()


dict_keys(['images', 'num_frames', 'offload_video_to_cpu', 'offload_state_to_cpu', 'video_height', 'video_width', 'device', 'storage_device', 'point_inputs_per_obj', 'mask_inputs_per_obj', 'cached_features', 'constants', 'obj_id_to_idx', 'obj_idx_to_id', 'obj_ids', 'output_dict', 'output_dict_per_obj', 'temp_output_dict_per_obj', 'consolidated_frame_inds', 'tracking_has_started', 'frames_already_tracked'])

In [11]:
inference_state["obj_ids"]

[]

In [26]:
import numpy as np
import cv2
cv2.imshow('img', img)
#np.array(img)[:, :, ::-1].shape
#plt.imshow(np.array(img)[:, :, :])

In [ ]:

with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    state = predictor.init_state(<your_video>)

    # add new prompts and instantly get the output on the same frame
    frame_idx, object_ids, masks = predictor.add_new_points_or_box(state, <your_prompts>):

    # propagate the prompts to get masklets throughout the video
    for frame_idx, object_ids, masks in predictor.propagate_in_video(state):